# SQL Subqueries - Lab

## Introduction

Now that you've seen how subqueries work, it's time to get some practice writing them! Not all of the queries will require subqueries, but all will be a bit more complex and require some thought and review about aggregates, grouping, ordering, filtering, joins and subqueries. Good luck!  

## Objectives

You will be able to:

* Write subqueries to decompose complex queries

## CRM Database Schema

Once again, here's the schema for the CRM database you'll continue to practice with.

<img src="images/Database-Schema.png" width="600">

## Connect to the Database

As usual, start by importing the necessary packages and connecting to the database **data.sqlite**.

In [2]:
#Your code here; import the necessary packages
import sqlite3
import pandas as pd
conn = sqlite3.connect('data.sqlite')
c = conn.cursor()

## Select the Average Number of Orders on a Product by Product Basis

In [3]:
#Your code here; use a subquery. No join will be necessary.
c.execute("""SELECT AVG(numOrders)
            FROM (SELECT productCode, count(distinct orderNumber) as numOrders
                    FROM products
                    JOIN orderdetails using(productCode)
                    GROUP BY productCode
                    ;""")
df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
print('Number of results:', len(df))
df.head()

OperationalError: near ";": syntax error

## Write an Equivalent Query using a Subquery

```SQL
select customerNumber,
       contactLastName,
       contactFirstName
       from customers
       join orders 
       using(customerNumber)
       where orderDate = '2003-01-31';
```

In [ ]:
#Your code here; use a subquery. No join will be necessary.

## Select the Total Number of Orders for Each Product Name

Sort the results by the total number of items sold for that product.

In [ ]:
#Your code here

## Select the Product Name and the  Total Number of People Who Have Ordered Each Product

Sort the results in descending order.

In [ ]:
#Your code here

## Select the Employee Number, Office Code, City (of the office), and Name (First and Last) of those Employees who Sold Products that Have Been Ordered by Less Then 20 people.

This problem is a bit tougher. To start, think about how you might break the problem up. Be sure that your results only list each employee once.

In [ ]:
#Your code here

## Select the Employee Number, Name (First and Last) and Number of Customers of Employees Who's Customers Have an Average Credit Limit of Over 15K

In [ ]:
#Your code here

## Summary

In this lesson, you got to practice some more complex SQL queries, some of which required subqueries. There's still plenty more SQL to be had though; hope you've been enjoying some of these puzzles!